# What Dis

2010–2016 crime data varaious cities throughout the United States as curated by some random website.

[source](http://www.city-data.com/crime/)

# Imports and Definitions

In [1]:
import pandas as pd
import numpy as np

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
def to_val(cell):
    val = cell.get_text().split('(')[0].strip().replace(',', '')
    return float(val) if val != 'N/A' else np.nan

In [4]:
places = {
    "Alabama": [
        "Birmingham",
        "Montgomery",],
    "Alaska": [
        "Anchorage",
        "Juneau",],
    "Arizona": [
        "Phoenix",
        "Tucson",],
    "Arkansas": [
        "Little Rock",
        "Fort Smith",],
    "California": [
        "San Diego",
        "Los Angeles",
        "Sacramento",],
    "Colorado": [
        "Denver",
        "Colorado Springs",
        "Aurora",],
    "Connecticut": [
        "Bridgeport",
        "Hartford",],
    "Delaware": [
        "Dover",
        "Wilmington",],
    "Florida": [
        "Jacksonville",
        "Miami",
        "Tallahassee",],
    "Georgia": [
        "Atlanta",
        "Savannah",],
    "Hawaii": [
        "Honolulu",
        "Kauai",
        "Maui",],
    "Idaho": [
        "Boise",
        "Meridian",],
    "Illinois": [
        "Chicago",
        "Springfield",],
    "Indiana": [
        "Fort Wayne",
        "Indianapolis",],
    "Iowa": [
        "Cedar Rapids",
        "Des Moines",],
    "Kansas": [
        "Topeka",
        "Wichita",],
    "Kentucky": [
        "Frankfort",
        "Louisville",],
    "Louisiana": [
        "Baton Rouge",
        "New Orleans",],
    "Maine": [
        "Augusta",
        "Portland",],
    "Maryland": [
        "Baltimore",
        "Annapolis",],
    "Massachusetts": [
        "Boston",
        "Worcester",],
    "Michigan": [
        "Detroit",
        "Lansing",],
    "Minnesota": [
        "Minneapolis",
        "St. Paul",],
    "Mississippi": [
        "Jackson",
        "Gulfport",],
    "Missouri": [
        "Kansas City",
        "Jefferson City",],
    "Montana": [
        "Billings",
        "Helena",],
    "Nebraska": [
        "Lincoln",
        "Omaha",],
    "Nevada": [
        "Carson City",
        "Las Vegas",
        "Reno",],
    "New Hampshire": [
        "Concord",
        "Manchester",],
    "New Jersey": [
        "Newark",
        "Trenton",],
    "New Mexico": [
        "Albuquerque",
        "Santa Fe",],
    "New York": [
        "Albany",
        "New York City",],
    "North Carolina": [
        "Charlotte",
        "Raleigh",],
    "North Dakota": [
        "Bismarck",
        "Fargo",],
    "Ohio": [
        "Columbus",
        "Cleveland",],
    "Oklahoma": [
        "Oklahoma City",
        "Tulsa",],
    "Oregon": [
        "Portland",
        "Salem",],
    "Pennsylvania": [
        "Harrisburg",
        "Philadelphia",],
    "Rhode Island": [
        "Providence",
        "Warwick",],
    "South Carolina": [
        "Charleston",
        "Columbia",],
    "South Dakota": [
        "Pierre",
        "Sioux Falls",],
    "Tennessee": [
        "Nashville",
        "Memphis",],
    "Texas": [
        "Austin",
        "Houston",],
    "Utah": [
        "Salt Lake City",
        "West Valley City",],
    "Vermont": [
        "Burlington",
        "Montpelier",],
    "Virginia": [
        "Richmond",
        "Virginia Beach",],
    "Washington": [
        "Olympia",
        "Seattle",],
    "West Virginia": [
        "Charleston",
        "Huntington",],
    "Wisconsin": [
        "Madison",
        "Milwaukee",],
    "Wyoming": [
        "Casper",
        "Cheyenne",],
}

In [5]:
corrections = { # little quirks
    "Boise": "Boise City",
    "New York City": "New York",
}

# Scraping

In [6]:
data = []
with requests.Session() as session:
    for state, cities in places.items():
        #print("scraping data for {state}:".format(state=state))
        for city in cities:
            #print("{city},".format(city=city)
            state_url = "http://www.city-data.com/crime/crime-{city}-{state}.html" \
                        .format(city=(corrections[city] if city in corrections else city).replace(' ', '-'),
                                state=state.replace(' ', '-'))
            city_soup = BeautifulSoup(session.get(state_url).content, 'html.parser')
            if city_soup.find(id="errormsg"):
                print("unable to find {city}, {state}; skipping"
                      .format(city=city,
                              state=state))
                continue
            table = city_soup.find(id="crimeTab")
            city_data = []
            city_data.append([cell.find('h4').get_text()
                              for cell
                              in table.find('thead').find_all('tr')[-1].find_all('th')][1:])
            for row in table.find('tbody').find_all('tr')[:-1]: # ignore footer
                city_data.append([to_val(cell)
                                  for cell
                                  in row.find_all('td')[1:]])
            data.append(pd.DataFrame({
                'state': state,
                'city': city,
                'year': city_data[0],
                'murder': city_data[1],
                'rape': city_data[2],
                'robbery': city_data[3],
                'assault': city_data[4],
                'burglary': city_data[5],
                'theft': city_data[6],
                'vehicle theft': city_data[7],
                'arson': city_data[8],
            }))
        
df = pd.concat(data, ignore_index=True).set_index(['state', 'city', 'year']).sort_index()

unable to find Honolulu, Hawaii; skipping
unable to find Kauai, Hawaii; skipping
unable to find Maui, Hawaii; skipping
unable to find Carson City, Nevada; skipping
unable to find Nashville, Tennessee; skipping
unable to find West Valley City, Utah; skipping


In [7]:
df.to_pickle("citydata_crime_data.pkl")

In [8]:
df

arson  assault  burglary  murder   rape  robbery  \
state   city       year                                                     
Alabama Birmingham 2002  211.0   1697.0    4389.0    65.0  239.0   1186.0   
                   2003  175.0   1706.0    4831.0    85.0  204.0   1352.0   
                   2004  142.0   1593.0    5156.0    59.0  240.0   1369.0   
                   2005  136.0   1675.0    4933.0   104.0  241.0   1429.0   
                   2006  228.0   1422.0    4813.0   104.0  220.0   1429.0   
                   2007  221.0   1396.0    4864.0    86.0  229.0   1609.0   
                   2008  134.0   1456.0    5153.0    82.0  212.0   1499.0   
                   2009  135.0   1399.0    5019.0    65.0  198.0   1150.0   
                   2011  123.0   1916.0    5806.0    54.0  182.0   1011.0   
                   2012  117.0   2035.0    4704.0    67.0  152.0    983.0   
                   2013    NaN   1642.0    4018.0    63.0  178.0    969.0   
                   2014  145.0   2084.0    3750.0    52.0  182.0   1051.0   
                   2015  145.0   2355.0    3146.0    79.0  159.0   1114.0   
        Montgomery 2002    NaN    629.0    3812.0    30.0  118.0    698.0   
                   2003    NaN    516.0    4092.0    18.0  115.0    658.0   
                   2004    NaN    574.0    3734.0    25.0  108.0    623.0   
                   2005   41.0    656.0    3528.0    29.0  109.0    847.0   
                   2006    NaN    397.0    3380.0    27.0   74.0    686.0   
                   2007   53.0    405.0    3317.0    46.0   54.0    652.0   
                   2008    NaN    348.0    3513.0    23.0   49.0    463.0   
                   2009    NaN    302.0    3092.0    31.0   85.0    453.0   
                   2010   50.0    316.0    3279.0    25.0   55.0    401.0   
                   2011   31.0    284.0    2885.0    31.0   38.0    354.0   
                   2012   64.0    283.0    3409.0    44.0   41.0    459.0   
                   2014    NaN    526.0    2548.0    35.0   35.0    450.0   
                   2015    NaN    578.0    2342.0    33.0   40.0    391.0   
                   2016    NaN    675.0    2293.0    30.0   62.0    448.0   
Alaska  Anchorage  2002  141.0   1067.0    1521.0    18.0  254.0    382.0   
                   2003  128.0   1143.0    1418.0    17.0  244.0    340.0   
                   2004   82.0   1555.0    1523.0    15.0  263.0    331.0   
...                        ...      ...       ...     ...    ...      ...   
Wyoming Casper     2002   22.0    116.0     505.0     2.0   14.0     16.0   
                   2003   21.0     91.0     464.0     4.0   15.0     15.0   
                   2004   11.0     88.0     501.0     0.0    7.0     13.0   
                   2005   21.0    112.0     525.0     2.0    7.0     14.0   
                   2006   14.0     98.0     441.0     0.0   20.0     12.0   
                   2007    7.0    108.0     504.0     2.0   20.0     22.0   
                   2008   31.0    109.0     368.0     1.0   30.0     19.0   
                   2009   11.0     75.0     411.0     0.0   20.0     12.0   
                   2010   22.0     80.0     382.0     2.0    6.0     20.0   
                   2011   17.0     70.0     287.0     1.0   10.0     22.0   
                   2012   18.0     69.0     363.0     3.0    2.0     12.0   
                   2013    6.0     64.0     294.0     2.0    7.0     15.0   
                   2014    8.0     67.0     238.0     2.0    4.0     11.0   
                   2015    6.0     59.0     212.0     1.0    8.0     17.0   
                   2016    6.0     61.0     255.0     0.0    2.0      7.0   
        Cheyenne   2002    8.0     65.0     219.0     2.0   15.0     26.0   
                   2003    6.0     63.0     258.0     0.0   23.0     24.0   
                   2004   14.0     68.0     324.0     0.0   20.0     16.0   
                   2005    7.0     64.0     272.0     4.0   12.0     17.0   
                   20